In [1]:
import requests
import json
import pandas as pd
import datetime as dt
from pprint import pprint
from collections import defaultdict

In [2]:
from config import conn_str  
from config import api_key
from dboperations import *  

In [3]:
conn = connect2db(conn_str)

Connection Succesful


In [4]:
# set flight_dates array for api parameter
start_date = dt.date(2021, 4, 24)
number_of_days = 3 #for testing assigned 3 will be past 3 months value

flight_dates = []

for day in range(number_of_days): 
    flight_date = (start_date - dt.timedelta(days = day)).isoformat()
    flight_dates.append(flight_date)

print(flight_dates) 

['2021-04-24', '2021-04-23', '2021-04-22']


In [31]:
# get aviationstack.com api response for flight dates
airlines = defaultdict(list)
flights = defaultdict(list)
airports = defaultdict(list)
#departures = defaultdict(list)
#arrivals = defaultdict(list)

for flight_date in flight_dates:
    
    params = {'access_key': api_key, 'arr_iata': 'LAX', 'flight_date': flight_date}
    
    api_link = requests.get('https://api.aviationstack.com/v1/flights', params) 
    response = api_link.json()
    
    for data in response['data']: 
        airlines['airline'].append(data['airline']['name'])
        airlines['iata'].append(data['airline']['iata'])
        airlines['icao'].append(data['airline']['icao'])
         
        flights['flight_number'].append(data['flight']['number'])
        flights['iata'].append(data['flight']['iata'])
        flights['icao'].append(data['flight']['icao'])  
        
        flights['departure_airport'].append(data['departure']['airport'])
        flights['departure_timezone'].append(data['departure']['timezone'])
        flights['departure_iata'].append(data['departure']['iata'])
        flights['departure_icao'].append(data['departure']['icao'])
        flights['departure_terminal'].append(data['departure']['terminal'])
        flights['departure_gate'].append(data['departure']['gate'])
        flights['departure_delay'].append(data['departure']['delay'])
        flights['departure_scheduled'].append(data['departure']['scheduled'])
        flights['departure_estimated'].append(data['departure']['estimated'])
        flights['departure_actual'].append(data['departure']['actual'])
        flights['departure_estimated_runway'].append(data['departure']['estimated_runway'])
        flights['departure_actual_runway'].append(data['departure']['actual_runway'])
        
        flights['arrival_airport'].append(data['arrival']['airport'])
        flights['arrival_timezone'].append(data['arrival']['timezone'])
        flights['arrival_iata'].append(data['arrival']['iata'])
        flights['arrival_icao'].append(data['arrival']['icao'])
        flights['arrival_terminal'].append(data['arrival']['terminal'])
        flights['arrival_gate'].append(data['arrival']['gate'])
        flights['arrival_delay'].append(data['arrival']['delay'])
        flights['arrival_scheduled'].append(data['arrival']['scheduled'])
        flights['arrival_estimated'].append(data['arrival']['estimated'])
        flights['arrival_actual'].append(data['arrival']['actual'])
        flights['arrival_estimated_runway'].append(data['arrival']['estimated_runway'])
        flights['arrival_actual_runway'].append(data['arrival']['actual_runway'])         

In [32]:
# pprint(list (flights.items()))  

In [33]:
# create dataframes from dictionaries
airlines_df = pd.DataFrame(airlines)
flights_df = pd.DataFrame(flights) 

flights_df.head()

,flight_number,iata,icao,departure_airport,departure_timezone,departure_iata,departure_icao,departure_terminal,departure_gate,departure_delay,...,arrival_iata,arrival_icao,arrival_terminal,arrival_gate,arrival_delay,arrival_scheduled,arrival_estimated,arrival_actual,arrival_estimated_runway,arrival_actual_runway
0,265,AS265,ASA265,Manzanillo,America/Mexico_City,ZLO,MMZO,1,HG,6.0,...,LAX,KLAX,6,67,NaN,2021-04-24T15:53:00+00:00,2021-04-24T15:53:00+00:00,2021-04-24T15:42:00+00:00,2021-04-24T15:42:00+00:00,2021-04-24T15:42:00+00:00
1,535,WN535,SWA535,Metropolitan Oak International,America/Los_Angeles,OAK,KOAK,2,26,7.0,...,LAX,KLAX,1,18A,NaN,2021-04-24T20:00:00+00:00,2021-04-24T20:00:00+00:00,2021-04-24T19:35:00+00:00,2021-04-24T19:35:00+00:00,2021-04-24T19:35:00+00:00
2,510,NK510,NKS510,Metropolitan Oak International,America/Los_Angeles,OAK,KOAK,1,7,9.0,...,LAX,KLAX,5,56,NaN,2021-04-24T19:59:00+00:00,2021-04-24T19:59:00+00:00,2021-04-24T19:44:00+00:00,2021-04-24T19:44:00+00:00,2021-04-24T19:44:00+00:00
3,4259,WN4259,SWA4259,Love Field,America/Chicago,DAL,KDAL,1,7,12.0,...,LAX,KLAX,1,18A,NaN,2021-04-24T22:10:00+00:00,2021-04-24T22:10:00+00:00,None,None,None
4,215,UA215,UAL215,Denver International,America/Denver,DEN,KDEN,None,B47,11.0,...,LAX,KLAX,7,75B,NaN,2021-04-24T13:10:00+00:00,2021-04-24T13:10:00+00:00,2021-04-24T12:59:00+00:00,2021-04-24T12:59:00+00:00,2021-04-24T12:59:00+00:00


In [75]:
# clean airline frame for table
airlines_df.reset_index(inplace = True) #need that for following loop 
airlines_df = airlines_df.rename(columns = {"index": "airline_id", "name": "airline"})  


In [76]:
airlines_df.head()

,level_0,airline_id,airline,iata,icao
0,0,0,Alaska Airlines,AS,ASA
1,1,1,Southwest Airlines,WN,SWA
2,2,2,Spirit Airlines,NK,NKS
3,3,3,Southwest Airlines,WN,SWA
4,4,4,United Airlines,UA,UAL


In [29]:
# airline insert df to table
insertvalues(conn, "airlines", airlines_df)

TypeError: can only concatenate str (not "IntegrityError") to str

In [73]:
# create new frame for airports
airports_d_df = flights_df[['departure_airport', 'departure_iata', 'departure_icao', 'departure_timezone']] 
airports_d_df = airports_d_df.rename(columns = {'departure_airport': 'airport', 'departure_timezone': 'timezone', \
                                                'departure_iata': 'iata', 'departure_icao': 'icao'}) 
airports_d_df = airports_d_df.drop_duplicates() #subset=['departure_airport'], keep = False

airports_a_df = flights_df[['arrival_airport', 'arrival_iata', 'arrival_icao', 'arrival_timezone']]
airports_a_df = airports_d_df.rename(columns = {'arrival_airport': 'airport', 'arrival_timezone': 'timezone', \
                                                'arrival_iata': 'iata', 'arrival_icao': 'icao'}) 
airports_a_df = airports_a_df.drop_duplicates()
 
airports_df = pd.concat([airports_a_df, airports_d_df])
airports_df.reset_index(inplace = True)
airports_df = airports_df.rename(columns = {"index": "airport_id"})

In [74]:
airports_df.head()

,airport_id,airport,iata,icao,timezone
0,0,Manzanillo,ZLO,MMZO,America/Mexico_City
1,1,Metropolitan Oak International,OAK,KOAK,America/Los_Angeles
2,3,Love Field,DAL,KDAL,America/Chicago
3,4,Denver International,DEN,KDEN,America/Denver
4,5,Dallas/Fort Worth International,DFW,KDFW,America/Chicago


In [77]:
# airport insert df to table
insertvalues(conn, "airports", airports_df)

TypeError: can only concatenate str (not "IntegrityError") to str

In [78]:
closeconnection(conn);

DB Connection Closed
